## Necessary Packages
Here we import the necessary third party packages

In [1]:
import cv2
import re
import numpy as np
import matplotlib.pyplot as plt
from shapely import Polygon, intersection, union
from skimage.measure import find_contours
import os
import random
from skimage.draw import polygon
from skimage.color import rgb2gray
from skimage import img_as_ubyte
import os

In [48]:
image = cv2.imread('../tattoo_images/2776427893_58405d2a94.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

## Defining Functions for showing masks
show_mask shows the mask of the image that was predicted
show_points shows the prompting points
show_box shows the bounding box

In [2]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


## Functions for data pipeline

In [6]:
annotation_file = "../tattoo_annotations.txt"
annotations = []
match_pattern =r'(.+\.jpg) (\[.+\])'
split_pattern = r'[;\s\t\n\]]'
result = {}
with open(annotation_file) as file:
    for input_str in file.readlines():
        match = re.match(match_pattern, input_str)
        if match:
            filename = match.group(1)
            coordinates_str = match.group(2)
            coordinates_list = re.split(split_pattern, coordinates_str)
            coordinates_list = [x.strip(r"\[\]") for x in coordinates_list if x != ""]
            result[filename] = coordinates_list
        else:
            print("No matches!")
results_evaluated = {}
for k,v in result.items():
    results_evaluated[k] = [eval(x) for x in v]
dirlist = os.listdir("../tattoo_images")
keylist = results_evaluated.keys()

In [ ]:
points = results_evaluated["2776427893_58405d2a94.jpg"]
np.array(points)

In [7]:
def polygon_to_mask(image_shape, polygon_points):
    '''

    :param image_shape: Shape of image
    :param polygon_points: Points that are inside the polygon
    :return: Binary mask of the polygon
    '''
    polygon_points = np.array(polygon_points)
    rr, cc = polygon(polygon_points[:, 1], polygon_points[:, 0], image_shape)
    mask = np.zeros(image_shape, dtype=np.uint8)
    mask[rr, cc] = 1
    return mask, rr, cc

def convert_rgb_to_binary_mask(rgb_mask):
    """
    Convert an RGB mask to a binary mask.

    Parameters:
    rgb_mask (np.array): RGB mask of shape (height, width, 3)

    Returns:
    np.array: Binary mask of shape (height, width)
    """
    # Convert RGB mask to grayscale
    gray_mask = rgb2gray(rgb_mask)
    # Threshold the grayscale image to get a binary mask
    binary_mask = img_as_ubyte(gray_mask > 0)

    return binary_mask

In [ ]:
mask_1, rr, cc = polygon_to_mask(image.shape, points)
mask_2 = convert_rgb_to_binary_mask(mask_1)
plt.imshow(mask_2)

In [8]:

def random_points(mask, no_true = 1, no_false = 0):
    '''
    Selects n random points inside the mask, and m points outside in the background
    :param mask: Binary mask in the shape of the image
    :param no_true: n
    :param no_false: m
    :return: numpy array of points
    '''
    tattoo_indices = np.argwhere(mask == 255)
    background_indices = np.argwhere(mask == 0)
    tattoo_points = tattoo_indices[np.random.choice(len(tattoo_indices), no_true, replace=False)]
    background_points = background_indices[np.random.choice(len(background_indices), no_false, replace=False)]
    tattoo_points = tattoo_points[:, [1,0]]
    background_points = background_points[:, [1,0]]
    return tattoo_points, background_points


## Defining functions for results

In [34]:
def iou(pred, ground_truth):
    pred = pred.astype(int)
    ground_truth = ground_truth.astype(int)
    pred = pred.flatten()
    ground_truth = ground_truth.flatten()
    print(pred)
    print(ground_truth)
    intersection = np.sum(pred*ground_truth)
    union = np.sum(pred) + np.sum(ground_truth)
    print(intersection)
    print(union)
    print(np.sum(pred*ground_truth))
    iou = intersection / union if union != 0 else 0
    return iou

In [32]:
def fpr_fnr(pred, ground_truth):
    pred = pred.astype(int)
    ground_truth = ground_truth.astype(int)
    pred = pred.flatten()
    ground_truth = ground_truth.flatten()
    tp = np.sum(pred * ground_truth)
    fp = np.sum(pred * (1-ground_truth))
    fn = np.sum((1 - pred) * ground_truth)
    tn = np.sum((1 - pred) * (1 - ground_truth))
    print(tp,fp,fn, tn)
    fpr = fp/(fp+tn) if fp+tn != 0 else 0
    fnr = fn / (fn+tp) if fn+tp != 0 else 0
    return fpr, fnr


## Image Reading


In [3]:
image = cv2.imread('../tattoo_images/2776427893_58405d2a94.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
mask = polygon_to_mask(image.shape, points)
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.imshow(mask, interpolation= "none", cmap="jet", alpha=0.95)
plt.axis('off')
plt.savefig("test.svg", format="svg")
plt.show()

In [13]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "../sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

NameError: name 'image' is not defined

In [ ]:
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()


[(216.049382716, 470.906579041),
 (219.416386083, 451.519986809),
 (249.158249158, 446.516995266),
 (232.323232323, 428.38115092),
 (232.323232323, 418.375167833),
 (242.424242424, 397.112453773),
 (253.086419753, 392.734836172),
 (258.698092031, 398.363201659),
 (265.432098765, 412.121428404),
 (269.36026936, 417.124419947),
 (267.676767677, 430.882646692),
 (267.115600449, 435.260264293),
 (291.245791246, 461.525969897),
 (292.929292929, 461.525969897),
 (296.296296296, 459.649848068),
 (296.296296296, 441.514003722),
 (301.346801347, 436.511012179),
 (315.375982043, 438.387134008),
 (322.671156004, 435.885638236),
 (318.74298541, 426.505029092),
 (299.663299663, 409.619932632),
 (292.368125701, 406.493062917),
 (276.655443322, 404.616941088),
 (265.432098765, 387.731844629),
 (311.447811448, 368.345252397),
 (300.785634119, 355.212399595),
 (281.144781145, 317.064589075),
 (273.849607183, 290.798883471),
 (278.900112233, 280.167526441),
 (274.971941639, 260.155560267),
 (263.7485970

(0.0, 0.0)

1.0

## Writing all the jpg-files to a txt file

In [239]:
path = "../DeMSI"
for diri in os.listdir(path):
    path_to_files = os.listdir(os.path.join(path,diri))
    with open(f"../DeMSI/{diri}/{diri}.txt", "w") as fi:
        print(diri)
        for file in path_to_files:
            fi.writelines(f"{file}\n")

split_1
split_2
split_3
split_4
split_5


## Now: get the polygons! NO the bounding box!!!

In [245]:
def get_bounding_box(value):
    first_elements = [t[0] for t in value]
    second_elements = [t[1] for t in value]

    max_first = max(first_elements)
    min_first = min(first_elements)
    max_second = max(second_elements)
    min_second = min(second_elements)
    return np.array([min_first, min_second, max_first, max_second])

array([130.75196408,  16.88509646, 335.57800224, 470.90657904])

## Workprocess
What we will do:
    - imread the picture
    - read the polygon lines from the txt file
    - calculate fpr, fnr, IoU for ALL images!
    - save the result in a txt or csv file
    - calculate mean of this all

In [35]:
def all_combine_save_number(true=1, false=1, bounding_box = False):
    io_list = []
    fpr_list = []
    fnr_list = []
    for key, value in results_evaluated.items():
        image = cv2.imread(f"../tattoo_images/{key}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask, rr, cc = polygon_to_mask(image.shape, value)
        mask = convert_rgb_to_binary_mask(mask)
        predictor.set_image(image)
        # Create Prompt Points
        if not bounding_box:
            tattoo, background = random_points(mask, true, false)
            input_points = np.concatenate((tattoo, background))
            input_label = np.concatenate((np.ones(true), np.zeros(false)))
            masks, scores, logits = predictor.predict(
                point_coords=input_points,
                point_labels=input_label,
                multimask_output=True,
            )
            max_mask = masks[np.argmax(scores)]
            io = iou(max_mask, mask)
            fpr, fnr = fpr_fnr(max_mask, mask)
            for i, (mask_pred, score) in enumerate(zip(masks, scores)):
                plt.figure(figsize=(10,10))
                overlay = image.copy()
                overlay[mask_pred == True] = (0,255,0)
                overlay[mask == True] = (255,0,0)
                image = cv2.addWeighted(overlay, 0.3, image, 0.7, 0)
                for point in input_points:
                    cv2.circle(image, (point[0], point[1]), 5, (0,0,255), -1)
                plt.imsave(f"../mask/{key}_mask_{i+1}_score_{score:.3f}.jpg", image)
            io_list.append(io)
            fpr_list.append(fpr)
            fnr_list.append(fnr)
            print(io, fpr, fnr)
    return(io_list, fpr_list, fnr_list)


io_list, fpr_list, fnr_list = all_combine_save_number()


[0 0 0 ... 1 0 0]
[0 0 0 ... 0 0 0]
320790
477855
320790
320790 -163725 0 30435
0.6713124274099884 1.2283367094305648 0.0
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
20078700
20285558
20078700
20078700 -19991692 119850 -19358
0.9898026960855599 0.9990326344694557 0.005933594243151118
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
69615
85302
69615
69615 -53928 0 150313
0.8161004431314623 -0.5595061472220781 0.0
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
24990
5541804
24990
24990 -23571 5515395 -5350314
0.004509361933406522 0.00438621220960255 0.9954894831315875
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
1116135
2338306
1116135
1116135 -1104704 1210740 -1055671
0.47732632084936705 0.5113482612972285 0.5203287671232877
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
3981570
4997307
3981570
3981570 -3965688 999855 -877737
0.7967431258475816 0.8187776212081327 0.20071666240081903
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
12391980
12581434
12391980
12391980 -12340736 138210 -1954
0.9849417800864353 0.999841687671002 0.011030159957670234


C:\Users\nweiand\AppData\Local\Temp\ipykernel_16716\2120409559.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10,10))


[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
8726355
8894176
8726355
8726355 -8559044 510 19679
0.9811313605667349 1.002304503906321 5.84402302545072e-05


KeyboardInterrupt: 

<Figure size 1000x1000 with 0 Axes>

...

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

In [ ]:
io_list